In [72]:
import sys
sys.path.append("../..")
# from sources import *
import os
import os.path
from IPython.display import clear_output
import time
from geo.countries import Countries
from geo.utils import extract_countries
from geo.data import country_alias
c = Countries()
c.load_wb()
from data.Frame import Frame
import requests
import urllib
import json
import requests
import json
from pymongo import MongoClient
import db

# Get all json files containing group definitions
def get_group_json_files():
    from pathlib import Path
    path = Path(os.getcwd())
    files = []
    for f in os.listdir(str(path.parent.parent) + '//geo///cache/groups//'):
        files.append(str(path.parent.parent) + '//geo//cache//groups//' + f)
    
    return files

# Do some cleaning and adjustments to make names uniform
def replace(file, name, aliases):
    for alias in aliases:
        with open(file, 'r', encoding='utf8') as f:
            data = f.read()
            se = ("%s%s%s" %("'", alias,"'"))
            data = data.replace(se, name)
            with open(file, 'w', encoding='utf8') as f:
                f.write(data)
                f.close()
                
for country in country_alias:
    for gfile in get_group_json_files():
        replace(gfile, country, country_alias[country])

# Read json file
def read_json_file(fi):
    with open(fi, encoding='utf-8') as f:
        d = json.load(f)
    return d

# Load country data from restcountries
def read_restcountries():
    url = 'https://restcountries.eu/rest/v2/all'
    response = urllib.request.urlopen(url)
    data = response.read().decode('utf-8')
    country_names = json.loads(data)
    return country_names

rest_countries = read_restcountries()

# Searc rest countries
def get_country(name):
    for c in rest_countries:
        if c['name'] == name:
            return c

In [76]:
client = MongoClient('stats.whereistomas.org', username='root', password='Root!0676')
db = client["stats"]
url = 'http://country.io/names.json'
response = urllib.request.urlopen(url)
data = response.read().decode('utf-8')
country_names = json.loads(data)

db["groups"].drop()

for f in get_group_json_files():    
    print ("Processing %s" % (f))
    if "json" in f:
        data = read_json_file(f)
        countries = [] 
        # Add additional country information
        for n in data['names']:
            countries.append(get_country(n))
            
        data['countries'] = countries
        
        db["groups"].insert_one(data)

Processing C:\Users\tomas\datascrapper2//geo//cache//groups//acd.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//acp.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//ag.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//aosis.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//ap.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//asean.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//au.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//brics.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//can.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//canz.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//caricom.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//cegpl.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//cplp.json
Processing C:\Users\tomas\datascrapper2//geo//cache//groups//cw.json
Processing